In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import subgraph, k_hop_subgraph, dense_to_sparse
from torch_geometric.nn import GCNConv
from sklearn.metrics import accuracy_score
import torch.nn.functional as FF
import warnings

/opt/anaconda3/envs/gnn/lib/python3.9/site-packages/torch_geometric/typing.py:54: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: dlopen(/opt/anaconda3/envs/gnn/lib/python3.9/site-packages/libpyg.so, 0x0006): Library not loaded: /Library/Frameworks/Python.framework/Versions/3.9/Python
  Referenced from: <2695749E-A917-3F14-B0DA-2A4914D2853D> /opt/anaconda3/envs/gnn/lib/python3.9/site-packages/libpyg.so
  Reason: tried: '/Library/Frameworks/Python.framework/Versions/3.9/Python' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/Python.framework/Versions/3.9/Python' (no such file), '/Library/Frameworks/Python.framework/Versions/3.9/Python' (no such file)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/opt/anaconda3/envs/gnn/lib/python3.9/site-packages/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: dlopen(/opt/anaconda3/envs/gnn

In [2]:
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
def load_dataset_Cora():
    dataset=Planetoid(root='tmp/Cora', name='Cora')
    return dataset

In [4]:
def load_dataset_PubMed():
    dataset=Planetoid(root='tmp/PubMed', name='PubMed')
    return dataset

In [5]:
class GCN_Extraction(nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCN_Extraction,self).__init__()
        self.conv1=GCNConv(num_features,16)
        self.conv2=GCNConv(16, num_classes)
        
    def forward(self, x, edge_index):
        x=self.conv1(x,edge_index)
        x=torch.relu(x)
        x=self.conv2(x,edge_index)
        return torch.log_softmax(x, dim=1)

In [6]:
class GCN_Victim(nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCN_Victim,self).__init__()
        self.conv1=GCNConv(num_features,128)
        self.conv2=GCNConv(128,64)
        self.conv3=GCNConv(64, 16)
        self.conv4=GCNConv(16,num_classes)

    def forward(self, x, edge_index):
        x=self.conv1(x,edge_index)
        x=torch.relu(x)
        x = self.conv2(x, edge_index)
        x=torch.relu(x)
        x=self.conv3(x,edge_index)
        x=torch.relu(x)
        x=self.conv4(x, edge_index)
        return torch.log_softmax(x, dim=1)

In [40]:
def train_Victim(model, data, epoches=400, lr=0.01, weight_decay=5e-4):
    optimizer=optim.Adam(model.parameters(),lr=lr, weight_decay=weight_decay)
    model.train()
    
    for epoch in range(epoches):
        optimizer.zero_grad()
        out=model(data.x, data.edge_index)
        loss=FF.nll_loss(out[data.train_mask],data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        #if epoch % 10 ==0:
            #print(f"Epoch {epoch+10}, Loss: {loss.item():.5f}")

In [41]:
def train_Extraction(model,features, adjacency_matrix, labels, epoches=500, lr=0.01, weight_decay=5e-4):
    optimizer=optim.Adam(model.parameters(),lr=lr, weight_decay=weight_decay)
    model.train()

    patience_counter = 0
    
    for epoch in range(epoches):
        optimizer.zero_grad()
        out=model(features, adjacency_matrix)
        loss=FF.nll_loss(out,labels)
        loss.backward()
        optimizer.step()
            
        #if epoch % 10 ==0:
            #print(f"Epoch {epoch+10}, Loss: {loss.item():.5f}")

In [66]:
# Function to extract a 2-hop subgraph from a larger graph.
# It samples a random node and retrieves its 2-hop neighborhood if its size is within the specified limits.
def get_2hop_subgraph(data, min_size=10, max_size=150):
    num_nodes=data.num_nodes

    while True:
        node_idx=torch.randint(0,num_nodes,(1,)).item()
        
        subset, edge_index,_,_=k_hop_subgraph(node_idx, 2, data.edge_index, relabel_nodes=True)

        if min_size<=subset.size(0)<=max_size:
            As=torch.zeros((subset.size(0),subset.size(0)))
            As[edge_index[0],edge_index[1]]=1

            Xs=data.x[subset]

            # Return the adjacency matrix, features, index of the center node, the mapping relationship between the index and the node
            return As.numpy(), Xs.numpy(), node_idx, subset

In [10]:
# Function to compute the prior distributions for both feature occurrence and feature count
def get_distribution(data,labels,num_features,num_classes):
    F=[] # List to store feature distribution for each class
    M=[] # List to store feature count distribution for each class
    
    for c in range(num_classes):
        class_nodes=data.x[labels==c]

        feature_counts=class_nodes.sum(dim=0).numpy()
        feature_distribution=feature_counts/feature_counts.sum()
        F.append(feature_distribution)

        num_features_per_node=class_nodes.sum(dim=1).numpy()
        feature_count_distribution=np.bincount(num_features_per_node.astype(int),minlength=num_features)
        M.append(feature_count_distribution/feature_count_distribution.sum())
        
    return F,M

In [11]:
# Function to generate a sample graph based on input feature (Fc) and count (Mc) distributions 
def GenerateSample(Fc, Mc, As):
    num_nodes=As.shape[0]
    Ac=torch.ones((num_nodes,num_nodes))
    Xc=torch.zeros(num_nodes, len(Fc))

    for i in range(num_nodes):
        m=np.random.choice(np.arange(len(Mc)),p=Mc)
        features=np.random.choice(len(Fc),size=m,replace=False,p=Fc)
        Xc[i,features]=1

    return Ac.numpy(), Xc.numpy()

In [12]:
# Function to sample subgraphs and combine them into a larger graph
def SubgraphSamplingAlgorithm(f,Gs,F, M, n, C):
    As, Xs=Gs
    num_nodes=As.shape[0]
    SA=[As]
    SX=[Xs]
    SL=[api_out_class(f,Xs,As)]

    for i in range(n):
        # For each class, generate and store a new sampled subgraph
        for c in range(C):
            Ac, Xc=GenerateSample(F[c],M[c],As)
            SA.append(Ac)
            SX.append(Xc)
            SL.append(api_out_class(f,Xc,Ac))

    AG_list=[dense_to_sparse(torch.tensor(a))[0] for a in SA]
    XG=torch.vstack([torch.tensor(x) for x in SX])

    SL=torch.tensor(SL,dtype=torch.long).view(-1)

    # Filter out invalid labels (negative labels) and trim the labels to match the feature matrix size
    valid_mask = SL >= 0
    SL = SL[valid_mask]
    SL = SL[:XG.shape[0]]

    # Combine the edge indices of all subgraphs, adjusting the node indices to avoid overlap
    AG_combined=torch.cat([edge_index+i*num_nodes for i, edge_index in enumerate (AG_list)], dim=1)

    return XG, AG_combined, SL

In [13]:
# Function to predict class labels for a graph using its feature matrix and adjacency matrix.
def api_out_class(f,features, adjacency_matrix):
    features_tensor=torch.tensor(features,dtype=torch.float)
    adjacency_tensor=torch.tensor(adjacency_matrix,dtype=torch.long)

    # If the adjacency matrix is in dense form, convert it to edge list format (sparse)
    if adjacency_tensor.ndim==2 and adjacency_tensor.shape[0]==adjacency_tensor.shape[1]:
        adjacency_tensor=torch.nonzero(adjacency_tensor, as_tuple=False).t()

    with torch.no_grad():
        logits=f(features_tensor, adjacency_tensor)
        predictions = torch.argmax(logits, dim=1)

    
    return predictions.numpy()

In [14]:
# Function to compute and return the raw logits (class scores) for a graph using its feature matrix and adjacency matrix.
def api_out_vector(features, adjacency_matrix):
    features_tensor=torch.tensor(features,dtype=torch.float)
    adjacency_tensor=torch.tensor(adjacency_matrix,dtype=torch.long)

    # If the adjacency matrix is in dense form (square matrix), convert it to a sparse edge list format
    if adjacency_tensor.ndim==2 and adjacency_tensor.shape[0]==adjacency_tensor.shape[1]:
        adjacency_tensor=torch.nonzero(adjacency_tensor, as_tuple=False).t()
        
    with torch.no_grad():
        logits=f(features_tensor, adjacency_tensor)

    return logits.numpy()

In [15]:
# Function to evaluate a model and return predictions for nodes based on a mask.
def api(model, data, mask):
    model.eval()
    out=model(data.x, data.edge_index)
    pred=out[mask].argmax(dim=1)

    return pred

In [16]:
# Function to evaluate how well the extracted model replicates the predictions of the victim model.
def eval(victim_model, extracted_model,data, mask):
    original_preds_labels=api(victim_model,data,mask)
    extracted_preds_labels=api(extracted_model,data,mask)
    score=accuracy_score(original_preds_labels, extracted_preds_labels)

    return score

In [30]:
# Function to evaluate the extraction model multiple times and return the average accuracy score.
def get_f(victim_model,extraction_model,data,mask,times=100):
    list_out=[]
    for i in range(times):
        pred=eval(victim_model,extraction_model,data,mask)
        list_out.append(pred)
    
    return sum(list_out)/len(list_out)

In [31]:
def setup_Victim(data,dataset):
    f=GCN_Victim(data.x.shape[1],dataset.num_classes)
    train_Victim(f,data)

    return f

In [19]:
def extract(f,data,dataset,samples,F,M):
    As, Xs, center_node, node_mapping=get_2hop_subgraph(data)
    Gs=(As,Xs)
    print(f"The index of the node is {center_node}, the size of subgraph is {Gs[0].shape,Gs[1].shape}")
    XG, AG, SL=SubgraphSamplingAlgorithm(f,Gs,F, M, n=samples, C=dataset.num_classes)

    g=GCN_Extraction(XG.shape[1], dataset.num_classes)
    train_Extraction(g,XG,AG,SL)
    fidelity=get_f(f,g,data,data.val_mask)
    print(f"The fidelity is {fidelity:.2f}")

In [20]:
def main_diffnodes(data,dataset,samples,F,M,extractTurns):
    f=setup_Victim(data,dataset)
    for i in range(extractTurns):
        print(f"For the {i+1}th extraction")
        extract(f,data,dataset,samples,F,M)

In [21]:
def main_diffsamples(data,dataset,samples,F,M):
    f=setup_Victim(data,dataset)
    As, Xs, center_node, node_mapping=get_2hop_subgraph(data)
    Gs=(As,Xs)
    print(f"The index of the node is {center_node}, the size of subgraph is {Gs[0].shape,Gs[1].shape}")
    for sample in samples:
        XG, AG, SL=SubgraphSamplingAlgorithm(f,Gs,F, M, n=sample, C=dataset.num_classes)
        print(f"{sample} Samples per class")
        g=GCN_Extraction(XG.shape[1], dataset.num_classes)
        train_Extraction(g,XG,AG,SL)
        fidelity=get_f(f,g,data,data.val_mask)
        print(f"The fidelity is {fidelity:.2f}")
        

In [50]:
#we dont need to care about extractTurns when the run_type is 'diffsamples'
def run_experiment(dataset_name,samples,extractTurns,run_type):
    if dataset_name == 'PubMed':
        dataset = load_dataset_PubMed()
        print("running on PubMed")
    elif dataset_name == 'Cora':
        dataset = load_dataset_Cora()
        print("running on Cora")
    else:
        raise ValueError("Invalid name, please use 'PubMed' or 'Cora'.")

    data=dataset[0]
    labels=data.y.numpy()
    num_classes=dataset.num_classes
    num_features=data.x.shape[1]

    F,M=get_distribution(data,labels,num_features,num_classes)
    
    if run_type == 'diffnodes' and isinstance(samples, int):
        main_diffnodes(data,dataset,samples,F,M,extractTurns)  
    elif run_type == 'diffsamples' and isinstance(samples, list):
        main_diffsamples(data,dataset,samples,F,M)  
    else:
        raise ValueError("Invalid params.")
    

In [ ]:
run_experiment('PubMEd',10,10,'diffnodes')

In [65]:
run_experiment('Cora',[1,3,5,10,100],10,'diffsamples')

running Cora
The index of the node is 1665, the size of subgraph is ((121, 121), (121, 1433))
1 Samples per class
The fidelity is 0.79
3 Samples per class
The fidelity is 0.81
5 Samples per class
The fidelity is 0.82
10 Samples per class
The fidelity is 0.83
100 Samples per class
The fidelity is 0.82


In [55]:
# For Algorithm 2
# TODO

In [ ]:
# Function to find the index of the center node in the subgraph.
def find_center_node_in_subgraph(center_global_idx, node_mapping):
    center_subgraph_idx = node_mapping.tolist().index(center_global_idx)
    return center_subgraph_idx

In [ ]:
# Function to approximate new nodes in the graph Gc based on KL divergence between nodes and the center node.
def approximate_nodes(Gc, api_out_vector, F, M, C, rho):
    A,X=Gc
    L=api_out_vector(X, A)
    center_idx=find_center_node_in_subgraph(center_node,node_mapping)
    
    for node in range(A.shape[0]):
        if node!=center_idx:
            for c in range(C):
                kl_div = np.exp(L[node][c]) * (L[node][c] - L[center_idx][c])
                
                if kl_div > 0:
                    num_new_nodes = int(kl_div * rho * np.log1p(A[node].sum())) # TODO
                    for _ in range(num_new_nodes):
                        new_node = A.shape[0]
                        
                        new_adj_row = np.zeros((1, A.shape[1]))
                        A = np.vstack([A, new_adj_row])
                        
                        new_adj_col = np.zeros((A.shape[0], 1))
                        A = np.hstack([A, new_adj_col])
                        
                        A[node, new_node] = 1
                        A[new_node, node] = 1
                        
                        feature_count = np.random.choice(len(M[c]), p=M[c])  
                        new_features = np.zeros_like(F[c])
                        chosen_features = np.random.choice(len(F[c]), size=feature_count, p=F[c])  
                        new_features[chosen_features] = 1 
                        
                        X = np.vstack([X, new_features])
    
    Gc=(A,X)
    return Gc

In [ ]:
Gs_with_approximated_nodes = approximate_nodes(Gs, api_out_vector, F, M, C=dataset.num_classes,rho=2)

In [ ]:
print(f"The size of tensors with algo1{Gs[0].shape,Gs[1].shape}")
print(f"The size of tensors with algo2{Gs_with_approximated_nodes[0].shape,Gs_with_approximated_nodes[1].shape}")

In [ ]:
XG1, AG1, SL1=SubgraphSamplingAlgorithm(Gs_with_approximated_nodes,F, M, n=10, C=dataset.num_classes)

In [ ]:
g1=GCN_Extraction(XG1.shape[1], dataset.num_classes)

In [ ]:
train_Extraction(g1,XG1,AG1,SL1)

In [ ]:
print(get_f(f,g1,data,data.val_mask,runturns))